In [13]:
# -*- coding: utf8 -*-

In [14]:
debug=False

filePrefix="MGC-160104"
nomLexique="/Users/gilles/Copy/Python/phonemisation/bdlexique-PDM-Resync2.txt"

freqForme=1000000

##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [15]:
import pandas as pd
import pickle

#Préparation du tableau de VERBES

###Lecture du lexique
- nomLexique pour le fichier
- names pour les noms de colonnes
- élimination des lignes dupliquées éventuelles (p.e. dépendre)

In [16]:
bdlexique=pd.read_csv(nomLexique,sep=";",names=["ortho","phono","ext","cs","ms","vs","lexeme","L23","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"],encoding="utf8")
bdlexique.loc[(bdlexique["ms"]=="ij")&(bdlexique["cs"].isin(["V","K"])),"ms"]="MS"
bdlexique.drop_duplicates(inplace=True,subset=["phono","ext","cs","ms","vs","lexeme","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"])

### Création de la colonne de fréquence *freq*

- fréquence forme : 1000000x(formeFrantext+formeFilms)
- fréquence lexeme : lemmeFrantext+lemmeFilms
- fréquence inexistante : .01

In [17]:
bdlexique["freq"]=1.0/100
filtreLFrantext=(bdlexique["lemmeFrantext"]!="***") 
filtreLFilms=(bdlexique["lemmeFilms"]!="***")
filtreFFrantext=(bdlexique["formeFrantext"]!="***") 
filtreFFilms=(bdlexique["formeFilms"]!="***")
bdlexique.loc[filtreLFrantext|filtreLFilms,"freq"]=bdlexique[filtreLFrantext]["lemmeFrantext"].astype(float)+bdlexique[filtreLFilms]["lemmeFilms"].astype(float)
bdlexique.loc[filtreFFrantext | filtreFFrantext,"freq"]=bdlexique[filtreFFrantext]["formeFrantext"].astype(float)*freqForme+bdlexique[filtreFFilms]["formeFilms"].astype(float)*freqForme

###Elimination des colonnes inutiles
- indicateur L23
- fréquences brutes

In [18]:
inutilesColonnes=["L23","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"]
for colonne in inutilesColonnes:
    del bdlexique[colonne]

###Extraction des formes verbales dans une structure *verbes*

In [19]:
verbes=bdlexique[bdlexique["cs"].isin(["V","K"])].copy()

In [20]:
verbes.loc[verbes["ms"]=="ij","ms"]="MS"

###Ajout d'une colonne *prob* pour la fréquence relative

In [21]:
verbes["prob"]=verbes["freq"]/verbes["freq"].sum()

###Ajout d'une colonne *case*
- remplir les *ms* vides pour permettre la concaténation sans erreur
- remplir les *vs* vides des participes passés par "pp" pour permettre l'identification de la case
- *case* est la concaténation de *vs* et *ms*

In [22]:
verbes["ms"]=verbes["ms"].fillna("")
verbes["vs"]=verbes["vs"].fillna("pp")
verbes["case"]=verbes["vs"]+verbes["ms"]

In [23]:
bdlexiqueIn = unicode(u"èò")
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO")
neutralise = dict(zip(bdlexiqueNum, neutreOut))

def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

verbes["phono"]=verbes["phono"].apply(recoder)

In [24]:
with open(filePrefix+'-Verbes2.pkl', 'wb') as output:
   pickle.dump(verbes, output, pickle.HIGHEST_PROTOCOL)